In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import locale
import re
import seaborn as sns
from IPython.display import display
import matplotlib.colors as mcolors

In [56]:
class DataAnalyze:
    # Klassenvariablen
    df_habit_cb = None
    df_habit_score = None
    df_usage = None
    
    def __init__(self):
        # Initialisierung der Klassenvariablen
        DataAnalyze.df_habit_cb = pd.read_csv(r"C:\Users\mobau\OneDrive\Dokumente\_Beruf\_Freizeit\HandyNutzung\CsvDateien\habit_checkbox_data.csv")
        DataAnalyze.df_habit_score = pd.read_csv(r"C:\Users\mobau\OneDrive\Dokumente\_Beruf\_Freizeit\HandyNutzung\CsvDateien\habit_score_data.csv")
        DataAnalyze.df_usage = pd.read_csv(r"C:\Users\mobau\OneDrive\Dokumente\_Beruf\_Freizeit\HandyNutzung\CsvDateien\usage_data.csv")
        self.__process_dataframes()
    
    def __process_dataframes(self):
        self.df_habit_score = self.df_habit_score.rename(columns = {'Unnamed: 0' : 'Habits'})
        self.df_habit_score = self.df_habit_score.set_index('Habits')
        self.df_habit_cb = self.df_habit_cb.rename(columns = {'Unnamed: 0' : 'Habits'})
        self.df_habit_cb = self.df_habit_cb.set_index('Habits')
        self.df_habit_cb = self.df_habit_cb.replace(-1,0).replace(2,1)
        self.df_usage = self.df_usage.set_index('Unnamed: 0')

    @staticmethod
    def plot_pie_chart(df, column_name='Gesamte Nutzung', threshold=0.01):
        # Trenne Einträge über und unter dem Schwellenwert
        main_data = df[df[column_name] >= threshold]
        other_data = df[df[column_name] < threshold]
        
        # Summiere kleine Werte und füge sie als 'Weitere' hinzu
        other_sum = other_data[column_name].sum()
        main_data = pd.concat([main_data, pd.DataFrame({column_name: [other_sum]}, index=['Weitere'])], ignore_index=False)
        
        # Plot des Kreisdiagramms
        plt.figure(figsize=(8, 8))
        plt.pie(main_data[column_name], labels=main_data.index, startangle=90,
                textprops={'fontsize': 9}, autopct=lambda p: f'{p:.1f}% ')
        plt.show()

    @staticmethod
    def group_columns_by_weekday(df, count=False):
        weekday_mapping = {col: pd.to_datetime(col).day_name() for col in df.columns}
        grouped_df = df.groupby(by=lambda x: weekday_mapping.get(x, x), axis=1).sum()
        
        if count: 
            counts = pd.Series(weekday_mapping).value_counts()
            grouped_df.loc["Gesamt"] = [counts.get(day, 0) for day in grouped_df.columns]
        
        return grouped_df

    @staticmethod
    def group_columns_by_month(df):
        month_mapping = {col: pd.to_datetime(col).strftime('%Y-%m') for col in df.columns}
        return df.groupby(by=lambda x: month_mapping.get(x, x), axis=1).sum()

    def get_compare_datenframes_lernen_diagramm(self):
        df = self.df_habit_score.loc['Lernen'].to_frame().T
        dftest = self.df_habit_cb.loc['Lernen'].to_frame().T
        df.loc['Checkmarks'] = dftest.loc['Lernen']
        df = df[sorted(df.columns)].loc[:, '2024-01-01':'2024-03-01']
        display(df)
        self._plot_linegraph(df.T, 'Compare Datenframes "Lernen"')

    def show_dataframes(self):
        pd.set_option('display.max_columns', 7)
        pd.set_option('display.max_rows', 7)
        print('Datenframe für die Gewohnheitscheckboxen: ')
        display(self.df_habit_cb)
        print('Datenframe für den Gewohnheitsscore: ')
        display(self.df_habit_score)
        print('Datenframe für Handydatennutzung: ')
        display(self.df_usage)

    def get_habits_barchart(self):
        df = self.df_habit_cb.copy()
        df['Summe'] = df.sum(axis=1)
        self._plot_bar_chart(df['Summe'], 'Anzahl ausgeführter Gewohnheiten', 'Anzahl')

    def get_linegraph_habits_score(self):
        df = self.df_habit_score[sorted(self.df_habit_score.columns)]
        df.columns = pd.to_datetime(df.columns, errors='coerce')
        self._plot_linegraph(df.T, 'Habits Score')

    def get_correlationmatrix_habits(self):
        df_gewohnheiten = self.df_habit_score.copy().T
        display(df_gewohnheiten.corr())

    def _plot_linegraph(self, df, title):
        ax = df.plot(kind='line', title=title)
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=10)
        plt.show()

    def _plot_bar_chart(self, data, title, ylabel):
        plt.figure(figsize=(10, 6))
        bars = data.plot(kind='bar', color='skyblue')
        for bar in bars.patches:
            height = bar.get_height()
            bars.text(bar.get_x() + bar.get_width() / 2, height, f'{height}', ha='center', va='bottom', fontsize=10)
        plt.title(title)
        plt.ylabel(ylabel)
        plt.xticks(rotation=0)
        plt.tight_layout()

    def plot_usage_intervals_with_groups(self):
        df = self.df_usage[self.df_usage['Gruppe'] != 'Gesamte Nutzung']
        df = df[df['Gesamte Nutzung'] < 0.02].drop(columns='Gesamte Nutzung')
        df.iloc[:, :-1] = (df.iloc[:, :-1] > 0).astype(int)  # Umwandlung auf 1, wenn > 0
        df['Anzahl Nutzung'] = df.iloc[:, :-1].sum(axis=1)
        self._plot_usage_intervals(df)

    def _plot_usage_intervals(self, df):
        intervals = list(range(0, 101, 10))
        color_map = dict(zip(df['Gruppe'].unique(), mcolors.TABLEAU_COLORS.values()))
        
        interval_group_counts = []
        usage_counts = []
    
        for i in range(len(intervals) - 1):
            lower, upper = intervals[i], intervals[i + 1]
            subset = df[(df['Anzahl Nutzung'] >= lower) & (df['Anzahl Nutzung'] < upper)]
            usage_counts.append(subset.shape[0])
            interval_group_counts.append(subset['Gruppe'].value_counts())
    
        last_subset = df[df['Anzahl Nutzung'] >= intervals[-1]]
        usage_counts.append(last_subset.shape[0])
        interval_group_counts.append(last_subset['Gruppe'].value_counts())
    
        self._plot_stacked_bars(interval_group_counts, usage_counts, color_map, intervals)

    def _plot_stacked_bars(self, interval_group_counts, usage_counts, color_map, intervals):
        fig, ax = plt.subplots()
        bottom = [0] * len(usage_counts)
    
        for i, group_counts in enumerate(interval_group_counts):
            for j, (group, count) in enumerate(group_counts.items()):
                ax.bar(i, count, bottom=bottom[i], color=color_map.get(group, 'gray'))
                bottom[i] += count
    
        for i, count in enumerate(usage_counts):
            ax.text(i, count + 0.1, str(count), ha='center', va='bottom')

        handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color_map[group], markersize=10)
                   for group in interval_group_counts[0].keys()]
        ax.legend(handles, list(interval_group_counts[0].keys()), title='Gruppen')
        ax.set_xlabel('Tage')
        ax.set_ylabel('Anzahl der Apps')
        ax.set_title('Nutzungshäufigkeit von Apps in Tagen (mit Gruppierung)')
        ax.set_xticks(range(len(usage_counts)))
        ax.set_xticklabels([f"{intervals[i]}-{intervals[i + 1] - 1}" for i in range(len(intervals) - 1)] + [f"{intervals[-1]}+"])
        plt.show()

    def get_linegraph_habit_year(self):
        df_habit_year = self.df_habit_cb[sorted(self.df_habit_cb.columns)]
        grouped_month = self.group_columns_by_month(df_habit_year).T
        self._plot_linegraph(grouped_month, 'Liniengraph für A')

    def get_linegraph_habit_week(self):
        df_gewohnheiten = self.df_habit_cb[sorted(self.df_habit_cb.columns)]
        grouped_df = self.group_columns_by_weekday(df_gewohnheiten)
        dfLine = grouped_df[['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']].T
        self._plot_linegraph(dfLine, 'Habits pro Wochentag')

    def get_piechart_app(self):
        df = self.df_usage[self.df_usage['Gruppe'] != 'Gesamte Nutzung']
        df = df.sort_values(by='Gesamte Nutzung', ascending=True)
        self.plot_pie_chart(df, 'Gesamte Nutzung', 0.02)

    def get_correlationmatrix_habit_usage(self):
        df = self.df_usage[self.df_usage['Gruppe'] != 'Gesamte Nutzung']
        df_grouped = df.groupby('Gruppe').sum().sort_values(by='Gesamte Nutzung', ascending=True)
        df_gewohnheiten = self.df_habit_score.copy()
        combinations = [('Kreatin', 'Training'), ('Kreatin', 'Lernen'), ('Lernen', 'Training'), ('Kreatin', 'Training', 'Lernen')]
        for combo in combinations:
            df_gewohnheiten.loc[f'{" & ".join(combo)}'] = df_gewohnheiten.loc[list(combo)].mean(axis=1)
        
        df_gewohnheiten = df_gewohnheiten.T
        common_dates = df_grouped.columns.intersection(df_gewohnheiten.index)
        df_handy = df_grouped[common_dates].T
        df_gewohnheiten = df_gewohnheiten.loc[common_dates]
        combined_df = pd.concat([df_gewohnheiten, df_handy], axis=1, join='inner')
        
        habit_columns = ['Kreatin', 'Lernen', 'Training', 'Kreatin & Training', 'Kreatin & Lernen', 'Lernen & Training', 'Kreatin & Training & Lernen']
        program_columns = [col for col in combined_df.columns if col not in habit_columns]
        
        correlation_matrix = combined_df[habit_columns + program_columns].corr()
        filtered_matrix = correlation_matrix.loc[habit_columns, program_columns]
        filtered_matrix = filtered_matrix[(filtered_matrix.abs() >= 0.4).any(axis=1)]
        filtered_matrix = filtered_matrix.loc[:, (filtered_matrix.abs() >= 0.4).any(axis=0)]
        
        self._plot_heatmap(filtered_matrix)

    def _plot_heatmap(self, matrix):
        plt.figure(figsize=(15, 10))
        sns.heatmap(matrix, annot=True, fmt=".2f", cmap="coolwarm", linewidths=0.5, annot_kws={"size": 25})
        plt.xticks(fontsize=30)
        plt.yticks(fontsize=30)
        plt.tight_layout()
        plt.show()

In [60]:
def main():
    ### Entferne die Kommentarzeichen, um die Funktionen auszuführen
    ### Initialisierung des Analyzers
    analyzer = DataAnalyze()

    ### Zeigt alle Datenframes an 
    # analyzer.show_dataframes()

    ### Zeigt das Diagramm zum Vergleich der Checkboxen und Scores der Gewohnheiten
    # analyzer.get_compare_datenframes_lernen_diagramm()

    ### Zeigt ein Balkendiagramm für die Anzahl der ausgeführten Gewohnheiten
    # analyzer.get_habits_barchart()

    ### Zeigt den Liniengraph für den Gewohnheitsscore
    # analyzer.get_linegraph_habits_score()

    ### Zeigt Nutzungshäufigkeit in Intervallen und Gruppen
    # analyzer.plot_usage_intervals_with_groups()

    ### Zeigt den Liniengraph für die jährliche Gewohnheitsauswertung
    # analyzer.get_linegraph_habit_year()

    ### Zeigt den Liniengraph für die wöchentliche Gewohnheitsauswertung
    # analyzer.get_linegraph_habit_week()

    ### Zeigt ein Kreisdiagramm für Apps basierend auf ihrer Nutzung
    # analyzer.get_piechart_app()

    ### Zeigt die Korrelationsmatrix zwischen den Gewohnheiten und der Nutzung
    # analyzer.get_correlationmatrix_habit_usage()

    ### Zeigt die Korrelationsmatrix der Gewohnheiten untereinander
    # analyzer.get_correlationmatrix_habits()

# Hauptfunktion aufrufen
if __name__ == "__main__":
    main()
